# CMA-MESO 1KM

CMA-MESO 基础 GRIB2 数据文件有 402 个要素场。

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

from reki.data_finder import find_local_file
from reki.format.grib.eccodes import load_field_from_file

In [ ]:
start_time = pd.Timestamp.utcnow().floor(freq="D") - pd.Timedelta(days=2)
start_time_label = start_time.strftime("%Y%m%d%H")
forecast_time_label = "24h"
forecast_time = pd.to_timedelta(forecast_time_label)

meso_grib2_orig_file_path = find_local_file(
    "cma_meso_1km/grib2/orig",
    start_time=start_time,
    forecast_time=forecast_time,
)
meso_grib2_orig_file_path

## 要素列表

使用 ecCodes 打印 CMA-GFS 基础 GRIB2 数据的要素列表。

In [ ]:
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load eccodes/2.29.0/intel && grib_ls -P count "{meso_grib2_orig_file_path}" 2>/dev/null

print(captured.stdout)

使用 wgrib2 打印 CMA-GFS 基础 GRIB2 数据的要素列表。

In [ ]:
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load wgrib2/3.1.1/intel && wgrib2 "{meso_grib2_orig_file_path}" 2>/dev/null

print(captured.stdout)

对 wgrib2 输出信息进行分组展示。

```{note}
本节代码由 ChatGPT 生成。
```

In [ ]:
import re
import textwrap
from collections import defaultdict
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load wgrib2/3.1.1/intel && wgrib2 "{meso_grib2_orig_file_path}" 2>/dev/null

text = captured.stdout

lines = text.strip().splitlines()
grouped = defaultdict(list)

pattern = re.compile(r'^(\d+):\d+:d=\d+:([^:]+):([^:]+):([^:]+):?$')

for line in lines:
    match = pattern.match(line)
    if match:
        index, param, level, fcst = match.groups()
        grouped[(param.strip(), fcst.strip())].append((level.strip(), int(index)))

WRAP_WIDTH = 80

for (param, fcst), level_index_list in grouped.items():
    level_index_list.sort(key=lambda x: x[1])
    level_strs = [f"{level} ({index})" for level, index in level_index_list]
    full_line = ', '.join(level_strs)
    wrapped = textwrap.fill(full_line, width=WRAP_WIDTH, subsequent_indent='  ', initial_indent='  ')
    print(f"{param} @ {fcst}:\n{wrapped}\n")